In [1]:
%load_ext watermark
%watermark  -d -u -a 'Alex S. Oliveira'

Alex S. Oliveira 
last updated: 2016-12-02


# Apredendo com diferentes preditores

Aprendendo com diferentes.

In [2]:
% matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

rcParams['figure.figsize'] = 15, 6

In [3]:
db = pd.read_csv("opec/data_05_11_16.csv", index_col=0, parse_dates=True)

db_q = db.resample("Q").mean()

In [4]:
%ls eia

Markets_interest.csv     PET_CRD_CRPDN_ADC_MBBL_M.xls
nonOECD_consumption.csv  saudi_production.csv
nonOPEC_production.csv   Supply_summary.csv
OECD_consumption.csv     table3c_M_199701-201712.csv
opec_spare_capacity.csv  table4a_M_201101-201712.csv


In [5]:
features = pd.read_csv("oil_features.csv", index_col=0, parse_dates=True)

features_drift = features.shift(1)
features_drift.head()

,y-o-y % OECD Consumption Change,non-OECD Consumption Growth,Saudi Production Change,Spare Capacity
2001-03-31,NaN,NaN,NaN,NaN
2001-06-30,1.57,2.33,0.58,1.62
2001-09-30,0.32,4.67,-0.18,2.71
2001-12-31,-0.57,0.90,-0.68,2.56
2002-03-31,-0.80,2.91,-1.19,3.43


## Preciso torna y em valores booleanos

* No caso do preço subir **True**
* No caso de descer **False**

In [6]:
db_logical  = db_q - db_q.shift(1)
db_logical["Logical"] = db_logical.Value > 0
db_logical.head()

,Value,Logical
Date,,
2003-03-31,NaN,False
2003-06-30,-4.574102,False
2003-09-30,1.552320,True
2003-12-31,1.360492,True
2004-03-31,1.979687,True


In [7]:
X = np.array(features["2003-01":"2016-06"])
y = np.array(db_logical.Logical["2003-01":"2016-06"])


print(X.shape)
print(y.shape)

(54, 4)
(54,)


In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import TimeSeriesSplit

scores = list()
gnb = GaussianNB()
tscv = TimeSeriesSplit(n_splits=12)
train_score = list()
test_score = list()

for train_index, test_index in tscv.split(X):
    train_score.append(gnb.fit(X[train_index,:], y[train_index]))
    test_score.append(gnb.fit(X[test_index,:], y[test_index]))
    
    

In [17]:
cross_val_score.

[GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None),
 GaussianNB(priors=None)]

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression


tscv = TimeSeriesSplit(n_splits=10)
train_score = list()
test_score = list()
for train_index, test_index in tscv.split(X):
        linear_regression = LinearRegression().fit(X[train_index,:], y[train_index])

In [ ]:
y_pred = linear_regression.predict(X)
linear_regression.get_params()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(3,1,1)

ax1.plot(y, color="b", marker="o", linestyle="--", label="Preco")
ax1.plot(y_pred, color = "r", marker="o", linestyle="-", label="Previsão")
ax1.set_ylabel("Preço")

ax1.legend(loc=1)


ax2 = fig.add_subplot(3,1,2)
perc = ((y_pred-y)/y)*100
ax2.bar(range(len(perc)),perc, align= "edge", label=r"$\frac{(yprevisto-y)}{y}$")
ax2.set_ylabel("(Previsto - Real)/Previsto")
ax2.set_title("Discrepância entre o preço previsto e o real.")
ax2.legend(loc=0, fontsize="xx-large")


ax3 = fig.add_subplot(3,1,3)

r_real=list()
r_pred = list()
for i in range(len(y[:])-1): 
    r_real.append((y[i+1][0]-y[i][0])/y[i])
    r_pred.append((y_pred[i+1][0]-y[i][0])/y[i])

ax3.bar(range(len(r_real)), r_real, color="b")
ax3.bar(range(len(r_pred)), r_pred, color="r")

ax3.set_ylabel("Retorno")

#fig.legend((price_line, prev_line),("Preço OPEC", "Preço previsto"), loc=1)

In [ ]:
r_pred = np.array(r_pred)
